In [263]:
import pandas as pd
import numpy as np
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
import spacy
import itertools as it

In [264]:
nlp = spacy.load('en')

In [265]:
nv_reviews = pd.DataFrame.from_csv("parsed_nv_reviews.csv", index_col = None)
# nv_reviews = nv_reviews.rename(index=str, columns={"review_text": "text"})
nv_reviews['parsed_text'] = None
nv_reviews.head(2)

# # check iterating through data frame

# i = 0
# for row in nv_reviews.text:
#     print row
#     i += 1
#     if i> 10:
#         break

,business_id,user_id,review_id,text,stars,useful,state,parsed_text
0,lKq4Qsz13FDcAVgp49uukQ,SnXZkRN9Yf060pNTk1HMDg,---3OXpexMp0oAg77xWfYA,Pizza here made my night... Good people and gr...,5,2.0,NV,None
1,Y-S2LhHefBdnX8VP5Bh_JA,mCa0WvcSCwtdfeUyhhfg8w,#VALUE!,I don't know why this place only has 3 stars b...,4,0.0,NV,None


In [266]:
#remove any encoding issues
# for i in range(1000):
#     nv_reviews.text[i] = nv_reviews.text[i].replace('\xe9', 'e')
#     nv_reviews.text[i] = nv_reviews.text[i].replace('\xf1', 'n')
#     nv_reviews.text[i] = nv_reviews.text[i].replace('\xe8', 'e')
#     nv_reviews.text[i] = nv_reviews.text[i].replace('\xc7', 'c')
#     nv_reviews.text[i] = nv_reviews.text[i].replace('\xc8', 'e')
1+1

2

In [267]:
%%time
if 0 == 1:

    # parse through the column
    i = 0
    for row in nv_reviews.text:
        nv_reviews['parsed_text'][i] = unicode(row, encoding = 'utf-8', errors = 'ignore')
        i += 1
    #     if i> 10:
    #         break
else:
    print('skipped')

C:\Users\skick\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Wall time: 2h 59s


In [268]:
nv_reviews.head(3)

,business_id,user_id,review_id,text,stars,useful,state,parsed_text
0,lKq4Qsz13FDcAVgp49uukQ,SnXZkRN9Yf060pNTk1HMDg,---3OXpexMp0oAg77xWfYA,Pizza here made my night... Good people and gr...,5,2.0,NV,Pizza here made my night... Good people and gr...
1,Y-S2LhHefBdnX8VP5Bh_JA,mCa0WvcSCwtdfeUyhhfg8w,#VALUE!,I don't know why this place only has 3 stars b...,4,0.0,NV,I don't know why this place only has 3 stars b...
2,3kdSl5mo9dWC4clrQjEDGg,tRXe5HRTDsUNqL3yNSquMw,#NAME?,2Nd time eating here today.1st time was great ...,1,0.0,NV,2Nd time eating here today.1st time was great ...


In [269]:
# for i in range(len(nv_reviews.parsed_text)):
#     nv_reviews['parsed'][i] = nlp(nv_reviews.parsed_text[i])

In [270]:
import nltk
from gensim import corpora
from collections import defaultdict
from pprint import pprint
import os
import codecs

# tokenize, lemmatize, n-gram


In [271]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [471]:
if 0 == 1:
    documents = nv_reviews['parsed_text'].tolist()

else:
    with open('docs.pkl', 'rb') as f:
        documents = pickle.load(f)

In [472]:
docs = []
for i in range(len(documents)):
    docs.append(nlp(documents[i]))

In [473]:
# docs

In [474]:
token_lemma = [token.lemma_ for token in docs[1]]

In [475]:
# filepaths
unigram_sentences_filepath = os.path.join('unigram.txt')
bigram_sentences_filepath = os.path.join('bigrams.txt')
bigram_model_filepath = os.path.join('bigram_model_all')
trigram_sentences_filepath = os.path.join('trigrams.txt')
trigram_model_filepath = os.path.join('trigram_model_all')
review_txt_filepath = os.path.join('review_text_all.txt')

In [476]:
review_count = 0

# create & open a new file in write mode
with codecs.open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:
    # if this review is not about a restaurant, skip to the next one
    # write the restaurant review as a line in the new file
    # escape newline characters in the original review text
    for i in range(len(nv_reviews['parsed_text'])):
        review_txt_file.write(nv_reviews['parsed_text'][i].replace('\n', '\\n') + '\n')
        review_count += 1

print u'''Text from {:,} restaurant reviews
          written to the new txt file.'''.format(review_count)

Text from 50,000 restaurant reviews
          written to the new txt file.


In [477]:
#unigrams
with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(review_txt_filepath):
        f.write(sentence + '\n')

In [478]:
unigram_sentences = LineSentence(unigram_sentences_filepath)


In [479]:
for unigram_sentence in it.islice(unigram_sentences, 430, 440):
    print u' '.join(unigram_sentence)
    print u''

so -PRON- friend and -PRON- go for the tacos

-PRON- get the barbacoa and be tell the mushroom be highly recommend

at 4.50 a pop -PRON- be expect some high quality stuff

but alas all -PRON- get be some pretty ho hum tacos on a nice look frosty plate

-PRON- friend order the cochinta and say -PRON- would come back for that again -PRON- also order off the brunch menu

-PRON- forget the name but -PRON- be akin to a glamorize version of nachos top with fried egg

extremely forgettable

-PRON- will give -PRON- this though service be amazing

these be some of the nice fellow ever and really make -PRON- feel quite comfortable

unfortunately -PRON- do not think the food be up to par



In [480]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 0 == 1:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

Wall time: 1.64 s


In [481]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 1:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

Wall time: 0 ns


In [482]:
bigram_sentences = LineSentence(bigram_sentences_filepath)


In [483]:
for bigram_sentence in it.islice(bigram_sentences, 430, 440):
    print u' '.join(bigram_sentence)
    print u''

so -PRON- friend and -PRON- go for the tacos

-PRON- get the barbacoa and be tell the mushroom be highly_recommend

at 4.50 a pop -PRON- be expect some high_quality stuff

but alas all -PRON- get be some pretty ho_hum tacos on a nice look frosty plate

-PRON- friend order the cochinta and say -PRON- would come back for that again -PRON- also order off the brunch menu

-PRON- forget the name but -PRON- be akin to a glamorize version of nachos top with fried egg

extremely forgettable

-PRON- will give -PRON- this though service be amazing

these be some of the nice fellow ever and really make -PRON- feel quite comfortable

unfortunately -PRON- do not think the food be up to par



In [484]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 0 == 1:

    trigram_model = Phrases(bigram_sentences)

    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

Wall time: 984 ms


In [485]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 1:

    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

Wall time: 0 ns


In [486]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [487]:
for trigram_sentence in it.islice(trigram_sentences, 410, 420):
    print u' '.join(trigram_sentence)
    print u''

because those other two girl do -PRON- maybe try put -PRON- in as like chip and salsa and a side of queso or something and -PRON- get an_attitude say no that there s absolutely no way to do that wow really

3 -PRON- finally bring out -PRON- chip and salsa and queso yes -PRON- pay the $_11 and -PRON- br -PRON- a tiny cup of salsa

-PRON- be so small -PRON- laugh and say be -PRON- serious normally -PRON- give -PRON- a normal sized container of salsa no this microscopic thing can -PRON- bring -PRON- a normal sized one consider -PRON- just pay eleven dollar for this and -PRON- bring out a big container of salsa and have an_attitude and do not say thank -PRON- or sorry or one dam thing to -PRON-

-PRON- be some mixed girl black and white or something with a tattoo on -PRON- neck

horrible

oh and for the record -PRON- have try -PRON- steak before and -PRON- be always sooo dry and taste blah

this place make -PRON- so mad

like other -PRON- go check out the stromboli that guy_fieri have a fo

In [488]:

trigram_reviews_filepath = os.path.join('trigram_transformed_reviews_all.txt')

In [489]:
# the STOPWORDS dictionary was moved within spacy for the newest version 
# spacy.en.language_data.STOP_WORDS

In [490]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 1:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in spacy.en.language_data.STOP_WORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

Wall time: 0 ns


In [491]:
print u'Original:' + u'\n'

for review in it.islice(line_review(review_txt_filepath), 10, 11):
    print review

print u'----' + u'\n'
print u'Transformed:' + u'\n'

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 10, 11):
        print review

Original:

This place is definitely for hipsters, which me and my boyfriend are not. It is well decorated and feels really "cool" but the food wasn't anything to rave about. One of those places that people go to just to say they've been there, I guess. I went for dinner, and the dinner menu is actually smaller than their lunch menu? I thought that was weird. All the things that I saw on yelp reviews and I actually wanted to try were not for sale at the time I went. I settled and had curry, it was alright. So they get an okay review, because everything there was okay. Nothing bad, just nothing special.

Oh I lied, there was one thing that was pretty great. We paid four bucks for their self-filtered water and it was actually really amazing. LOL. I know that sounds like I totally got ripped off but the water was literally tasteless it felt so weird to drink. Quite the experience. I wouldn't go back for water, but I am glad I tried it once!

----

Transformed:

place definitely hipster -PR

# LDA

In [492]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import cPickle as pickle

In [493]:
trigram_dictionary_filepath = os.path.join('trigram_dict_all.dict')

In [494]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if 0 == 0:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=50, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

Wall time: 8.48 s


In [495]:
trigram_bow_filepath = os.path.join('trigram_bow_corpus_all.mm')

In [496]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [497]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if 0 == 1:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

Wall time: 449 ms


In [500]:
lda_model_filepath = os.path.join('lda_model_all')

In [ ]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if 0 == 0:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=50,
                           id2word=trigram_dictionary,
                           workers=3,
                           iterations = 50)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

In [ ]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print u'{:20} {}'.format(u'term', u'frequency') + u'\n'

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print u'{:20} {:.3f}'.format(term, round(frequency, 3))

In [ ]:
explore_topic(topic_number=14)

In [ ]:
# skipped a few blocks

In [ ]:
# select top 50 words for each of the 20 LDA topics
top_words = [[word for _, word in lda_model.show_topic(topicno, topn=50)] for topicno in range(lda_model.num_topics)]
print(top_words)

In [ ]:
# get all top 50 words in all 20 topics, as one large set
all_words = set(itertools.chain.from_iterable(top_words))

print("Can you spot the misplaced word in each topic?")

# for each topic, replace a word at a different index, to make it more interesting
replace_index = np.random.randint(0, 10, lda_model.num_topics)

replacements = []
for topicno, words in enumerate(top_words):
    other_words = all_words.difference(words)
    replacement = np.random.choice(list(other_words))
    replacements.append((words[replace_index[topicno]], replacement))
    words[replace_index[topicno]] = replacement
    print("%i: %s" % (topicno, ' '.join(words[:10])))

In [ ]:
LDAvis_data_filepath = os.path.join('ldavis_prepared')

In [ ]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 0:

    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary)

    with open(LDAvis_data_filepath, 'w') as f:
        pickle.dump(LDAvis_prepared, f)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath) as f:
    LDAvis_prepared = pickle.load(f)

In [ ]:
# The LDA Visualization
pyLDAvis.display(LDAvis_prepared)

# Do More with LDA

In [ ]:
def get_sample_review(review_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review(review_txt_filepath),
                          review_number, review_number+1))[0]

In [ ]:
def lda_description(review_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the review text with spaCy
    parsed_review = nlp(review_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_review
                      if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_review = bigram_model[unigram_review]
    trigram_review = trigram_model[bigram_review]
    
    # remove any remaining stopwords
    trigram_review = [term for term in trigram_review
                      if not term in spacy.en.language_data.STOP_WORDS]
    
    # create a bag-of-words representation
    review_bow = trigram_dictionary.doc2bow(trigram_review)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    
    # sort with the most highly related topics first
    review_lda = sorted(review_lda, key=lambda (topic_number, freq): -freq)
    
    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print '{:25} {}'.format(topic_number,
                                round(freq, 3))

In [ ]:
sample_review = get_sample_review(50)
print sample_review

In [ ]:
lda_description(sample_review)

In [ ]:
sample_review = get_sample_review(100)
print sample_review

In [ ]:
lda_description(sample_review)

# Word Vector Embedding

In [ ]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join('word2vec_model_all')

In [ ]:
# %%time

# this is a bit time consuming - make the if statement True
# if you want to train the word2vec model yourself.
if 0 == 1:

    # initiate the model and perform the first epoch of training
    food2vec = Word2Vec(trigram_sentences, size=100, window=5,
                        min_count=20, sg=1, workers=4)
    
    food2vec.save(word2vec_filepath)

    # perform another 11 epochs of training
    for i in range(1,12):

        food2vec.train(trigram_sentences,
                       total_examples=food2vec.corpus_count,
                       epochs=food2vec.iter)
        food2vec.save(word2vec_filepath)
        
# load the finished model from disk
food2vec = Word2Vec.load(word2vec_filepath)
food2vec.init_sims()

print u'{} training epochs so far.'.format(food2vec.train_count)

In [ ]:
print u'{:,} terms in the food2vec vocabulary.'.format(len(food2vec.wv.vocab))

In [ ]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in food2vec.wv.vocab.iteritems()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda (term, index, count): -count)

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(food2vec.wv.syn0norm[term_indices, :],
                            index=ordered_terms)

# word_vectors   #prints the df

In [ ]:
def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in food2vec.most_similar(positive=[token], topn=topn):

        print u'{:20} {}'.format(word, round(similarity, 3))

In [ ]:
get_related_terms(u'salad')

In [ ]:
get_related_terms(u'pasta', topn=20)

In [ ]:
get_related_terms(u'man')

In [ ]:
get_related_terms(u'ice_cream')

get_related_terms(u'vanilla_ice_cream')

In [ ]:
get_related_terms(u'john')

In [ ]:
get_related_terms(u'signature')

In [ ]:
get_related_terms(u'cheeseburger')

In [ ]:
get_related_terms(u'lox')

In [ ]:
get_related_terms(u'vegas')

In [ ]:
get_related_terms(u'rude')

In [ ]:
get_related_terms('summer')

# Word Algebra

In [ ]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = food2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print term

In [ ]:
word_algebra(add=[u'lunch', u'night'], subtract=[u'day'])

In [ ]:
word_algebra(add=[u'tacos', u'italian'], subtract=[u'mexican'])
word_algebra(add=[u'bun', u'mexican'], subtract=[u'american'])
word_algebra(add=[u'filet_mignon', u'seafood'], subtract=[u'beef'])

In [ ]:
word_algebra(add=[u'fork', u'soup'])
word_algebra(add=[u'pancake', u'egg'])
word_algebra(add=[u'beef', u'bacon'])
word_algebra(add=[u'mcdonalds', u'expensive'])
word_algebra(add=[u'mcdonalds', u'fancy'])
word_algebra(add=[u'ihop', u'fancy'])
word_algebra(add=[u'los_angeles', u'nevada'], subtract = [u'california'])
word_algebra(add =[u'fruit', u'chocolate'])
word_algebra(add=['manager', 'rude'])

In [ ]:
word_algebra(add=['drink', 'barley'])
word_algebra(add=['water', 'caffeine'])
word_algebra(add=['cheap', 'alcohol'])
word_algebra(add=['cheap', 'martini'])
word_algebra(add=['margarita'], subtract=['mexican'])
word_algebra(add=['whiskey','soda'])

In [ ]:
food2vec.wv.vocab[12]

In [ ]:
documents[40008]

In [ ]:
mylist = ['I wish to complain about this parrot what I purchased not half an hour ago from this very boutique.', "Oh yes, the, uh, the Norwegian Blue...What's,uh...What's wrong with it?", "I'll tell you what's wrong with it, my lad. 'E's dead, that's what's wrong with it!", "No, no, 'e's uh,...he's resting."]

import pickle

with open('parrotss.pkl', 'wb') as f:
    pickle.dump(mylist, f)

In [ ]:
import pickle
with open('parrotss.pkl', 'rb') as f:
    mynewlist = pickle.load(f)
    
mynewlist

In [ ]:
import pickle
with open('docs2.pkl', 'wb') as f:
    pickle.dump(documents, f)

In [ ]:
with open('docs.pkl', 'rb') as f:
    doc2 = pickle.load(f)
    
doc2[40008]

In [ ]:
model.wv.doesnt_match("breakfast cereal dinner lunch".split())

In [ ]:
model.score(["The fox jumped over a lazy dog".split()])

In [ ]:
most_similar_cosmul(positive=[], negative=[], topn=10)